# Qrack Clifford+RZ (Measurements)and Mitiq

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

You also need the `mitiq` package. `unitaryfund/mitiq` is a Python package for error mitigation. It relies intermediate representations of circuits like Qiskit's `QuantumCircuit` or Cirq's `Circuit`. In the example below, we use Qiskit's `QuantumCircuit`.

In [1]:
width = 36
depth = 36
shots = 1024

In [2]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install pyqrack

import os
import collections
import math
import numpy as np
import random

from pyqrack import QrackSimulator, Pauli
from mitiq import zne
from qiskit.circuit.quantumcircuit import QuantumCircuit
from qiskit.compiler import transpile
from mitiq.zne.scaling.folding import fold_global
from mitiq.zne.inference import RichardsonFactory

# Change these according to your OpenCL devices and system specifications:
os.environ['QRACK_MAX_PAGE_QB']='27'
os.environ['QRACK_MAX_ALLOC_MB']='15872'
os.environ['QRACK_MAX_PAGING_QB']='30'
os.environ['QRACK_MAX_CPU_QB']='-1'
# os.environ['QRACK_QUNITMULTI_DEVICES']='1'
# os.environ['QRACK_QPAGER_DEVICES']='1'
os.environ['QRACK_NONCLIFFORD_ROUNDING_THRESHOLD']='1'

In [3]:
def cx(circ, q1, q2):
    circ.cx(q1, q2)

def cy(circ, q1, q2):
    circ.cy(q1, q2)

def cz(circ, q1, q2):
    circ.cz(q1, q2)

def acx(circ, q1, q2):
    circ.x(q1)
    circ.cx(q1, q2)
    circ.x(q1)

def acy(circ, q1, q2):
    circ.x(q1)
    circ.cy(q1, q2)

def acz(circ, q1, q2):
    circ.x(q1)
    circ.cz(q1, q2)
    circ.x(q1)

def swap(circ, q1, q2):
    circ.swap(q1, q2)

def nswap(circ, q1, q2):
    circ.cz(q1, q2)
    circ.swap(q1, q2)
    circ.cz(q1, q2)

def pswap(circ, q1, q2):
    circ.cz(q1, q2)
    circ.swap(q1, q2)   

def mswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.cz(q1, q2)

def iswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.cz(q1, q2)
    circ.s(q1)
    circ.s(q2)

def iiswap(circ, q1, q2):
    circ.sdg(q2)
    circ.sdg(q1)
    circ.cz(q1, q2)
    circ.swap(q1, q2)

def random_circuit(width, depth):
    circuit = QuantumCircuit(width)
    
    # Nearest-neighbor couplers:
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    row_len = math.ceil(math.sqrt(width))
    two_bit_gates = swap, pswap, mswap, nswap, iswap, iiswap, cx, cy, cz, acx, acy, acz

    for i in range(depth):
        # One unit of "magic" per layer:
        magic_bit = random.randrange(width)
        magic_axis = random.randrange(3)
        # Single bit gates
        for j in range(width):
            for p in range(3):
                # We're trying to cover 3 Pauli axes
                # with Euler angle axes x-z-x. 
                circuit.h(j)

                # We can trace out a quarter rotations around the Bloch sphere with stabilizer.
                rnd = random.randint(0, 3)
                if rnd & 1:
                    circuit.s(j)
                if rnd & 2:
                    circuit.z(j)

                # For each circuit layer, there is one unit of "magic."
                if (j == magic_bit) and (p == magic_axis):
                    angle = random.uniform(0, math.pi / 2)
                    circuit.rz(angle, j)

        # Fully-connected couplers:
        ###########################
        unused_bits = [list(range(width))]
        random.shuffle(unused_bits)
        while len(unused_bits) > 1:
            b1 = unused_bits.pop()
            b2 = unused_bits.pop()
            g = random.choice(two_bit_gates)
            g(circuit, b1, b2)
    
    return circuit

Run this next cell several times to produce a noisy value, (optionally tuning the width, depth, and magic settings above,) and run the last cell to mitigate with Mitiq.

In [4]:
def logit(x):
    # Theoretically, these limit points are "infinite,"
    # but precision caps out between 36 and 37:
    if x > (1 - 5e-17):
        return 37
    # For the negative limit, the precisions caps out
    # between -37 and -38
    elif x < 1e-17:
        return -38
    return max(-38, min(37, np.log(x / (1 - x))))

def expit(x):
    # Theoretically, these limit points are "infinite,"
    # but precision caps out between 36 and 37:
    if x >= 37:
        return 1.0
    # For the negative limit, the precisions caps out
    # between -37 and -38
    elif x <= -38:
        return 0.0
    return 1 / (1 + np.exp(-x))

def execute(circuit):
    """Returns the mirror circuit expectation value for unsigned integer overall bit string."""

    qsim = QrackSimulator(circ.width(), isSchmidtDecompose=False)
    qsim.run_qiskit_circuit(circ, 0)

    qubits = list(range(width))

    # This is the average integer bit string produced, as a fraction of maximum:
    p = sum(qsim.measure_shots(qubits, shots)) / (shots * (1 << circ.width()))

    # So as not to exceed floor at 0.0 and ceiling at 1.0, (assuming 0 < p < 1,)
    # we mitigate its logit function value (https://en.wikipedia.org/wiki/Logit)
    return logit(p)

In [5]:
circ = random_circuit(width, depth)
print("Raw gate count: ", sum(dict(circ.count_ops()).values()))

# basis_gates=["rz", "x", "y", "z", "sx", "sy", "s", "sdg", "cx", "cy", "cz", "swap"]
# circ = transpile(circ, basis_gates=basis_gates, optimization_level=3)
# print("Optimized gate count: ", sum(dict(circ.count_ops()).values()))

Raw gate count:  7825


In [6]:
%%time

qsim = QrackSimulator(width)
raw_value = expit(execute(circ)) * (1 << width)

print("Raw value: ", raw_value)

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
Raw value:  2878544491.6650386
CPU times: user 1.58 s, sys: 2.67 s, total: 4.25 s
Wall time: 4.33 s


In [7]:
%%time

scale_count = 5
max_scale = 5
factory = RichardsonFactory(scale_factors=[(1 + (max_scale - 1) * x / scale_count) for x in range(0, scale_count)])
mitigated_value = expit(zne.execute_with_zne(circ, execute, factory = factory, scale_noise = fold_global)) * (1 << width)

print("Raw value: ", raw_value)
print("Value with ZNE: ", mitigated_value)
print("Width/depth/magic: ", width, "/", depth, "/", depth)

Raw value:  2878544491.6650386
Value with ZNE:  3525523741.1571164
Width/depth/magic:  36 / 36 / 36
CPU times: user 29.4 s, sys: 13.6 s, total: 43 s
Wall time: 43.3 s
